In [23]:
ls /run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=admin,volume=Paltas_DataBase/Data_Base/arbol_2_fila_G

PhantomRGB/


In [2]:
DB="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=admin,volume=Paltas_DataBase/Data_Base"

In [19]:
import os
#import skimage
import torch
import numpy as np

In [39]:
Tr=os.listdir(DB)
im_t=['PhantomRGB', 'SenteraRGB', 'SenteraNIR']

In [18]:
Tr_ID=100
print(os.listdir(os.path.join(DB,Tr[Tr_ID],im_t[0])))
print(os.listdir(os.path.join(DB,Tr[Tr_ID],im_t[1])))
print(os.listdir(os.path.join(DB,Tr[Tr_ID],im_t[2])))

['23_julio_2__arbol_15_fila_E.JPG', '23_julio_1__arbol_15_fila_E.JPG', '14_julio_1__arbol_15_fila_E.JPG', '9_julio__arbol_15_fila_E.JPG', '19_agosto__arbol_15_fila_E.JPG', '5_agosto__arbol_15_fila_E.JPG', '2_julio__arbol_15_fila_E.JPG', '24_setiembre__arbol_15_fila_E.JPG', '23_junio_1__arbol_15_fila_E.JPG', '16_junio_2__arbol_15_fila_E.JPG', '26_mayo_2__arbol_15_fila_E.JPG', '29_marzo_2__arbol_15_fila_E.JPG', '19_mayo_1__arbol_15_fila_E.JPG', '11_junio_1__arbol_15_fila_E.JPG', '12_mayo_1__arbol_15_fila_E.JPG', '26_mayo_1__arbol_15_fila_E.JPG', '2_junio__arbol_15_fila_E.JPG', '12_mayo_2__arbol_15_fila_E.JPG', '13_agosto__arbol_15_fila_E.JPG', '29_marzo_1__arbol_15_fila_E.JPG', '28_abril_1__arbol_15_fila_E.JPG', '14_abril_2__arbol_15_fila_E.JPG', '28_abril_2__arbol_15_fila_E.JPG', '19_mayo_2__arbol_15_fila_E.JPG', '14_abril_1__arbol_15_fila_E.JPG', '7_mayo__arbol_15_fila_E.JPG', '16_junio_1__arbol_15_fila_E.JPG']
['24_setiembre_1__arbol_15_fila_E.JPG', '13_agosto_2__arbol_15_fila_E.JPG',

In [37]:
for Tr in os.listdir(DB):
    print(Tr)
    for Im_t in os.listdir(os.path.join(DB,Tr)):
        #print(Im_t)
        print(len((os.listdir(os.path.join(DB,Tr,Im_t)))))
        #for date in os.listdir(os.path.join(DB,Tr,Im_t)):
        #    print(os.listdir(os.path.join(DB,Tr,Im_t,date)))
        #    if len(date)==0:
        #        print(Tr)
        #        print(Im_t)
        #        print(len(date))
        #        print("cero")

arbol_1_fila_A
20
5
5
arbol_1_fila_B
28
5
5
arbol_1_fila_C
26
5
5
arbol_1_fila_D
28
5
5
arbol_1_fila_E
27
5
5
arbol_1_fila_F
28
4
4
arbol_1_fila_G
28
4
4
arbol_2_fila_A
5
24
5
arbol_2_fila_B
27
5
5
arbol_2_fila_C
27
5
5
arbol_2_fila_D
28
5
5
arbol_2_fila_E
5
27
5
arbol_2_fila_F
4
28
4
arbol_2_fila_G
9
0
0
arbol_3_fila_A
5
24
5
arbol_3_fila_B
5
27
5
arbol_3_fila_C
27
5
5
arbol_3_fila_D
27
5
5
arbol_3_fila_E
27
5
5
arbol_3_fila_F
28
4
4
arbol_3_fila_G
28
4
4
arbol_4_fila_A
5
23
5
arbol_4_fila_B
27
5
5
arbol_4_fila_C
27
5
5
arbol_4_fila_D
5
27
5
arbol_4_fila_E
28
5
5
arbol_4_fila_F
4
4
28
arbol_4_fila_G
28
4
4
arbol_5_fila_A
24
5
5
arbol_5_fila_B
27
5
5
arbol_5_fila_C
5
5
26
arbol_5_fila_D
27
5
5
arbol_5_fila_E
5
5
28
arbol_5_fila_F
4
4
26
arbol_5_fila_G
28
4
4
arbol_6_fila_A
5
25
5
arbol_6_fila_B
28
5
5
arbol_6_fila_C
27
5
5
arbol_6_fila_D
28
5
5
arbol_6_fila_E
28
5
5
arbol_6_fila_F
27
4
4
arbol_6_fila_G
28
4
4
arbol_7_fila_A
5
21
5
arbol_7_fila_B
28
5
5
arbol_7_fila_C
5
27
5
arbol_7_fil

In [124]:
Tr=os.listdir(DB)
ImType=['PhantomRGB', 'SenteraRGB', 'SenteraNIR']
ImType=np.array(ImType)
Tr=np.array(Tr)
IMAGES=np.vectorize(lambda DB,Tr,Type:np.array(os.listdir(os.path.join(DB,Tr,Type))),
             otypes=[object],
             signature="(),(),()->()")(DB,Tr,ImType.reshape(-1,1))

In [125]:
inp=list(Tr[5:7])
inp=np.array(inp)
Tr=Tr[np.vectorize(lambda Tr,i:np.array(np.where(Tr==i)),signature="(j),()->()")(Tr,inp)]

IMAGES=np.vectorize(lambda DB,Tr,Type:np.array(os.listdir(os.path.join(DB,Tr,Type))),
             otypes=[object],
             signature="(),(),()->()")(DB,Tr,ImType.reshape(-1,1))

inpd=['14_abril','29_marzo','16_junio']


In [134]:
#np.vectorize(lambda iarr,d:iarr.split("__")[0])(IMAGES[0][0],np.array(inpd))
#np.vectorize(lambda iarr:iarr.split("__")[0])(IMAGES[0][0])
parsedate=np.vectorize(lambda iarr:("_").join(iarr.split("_")[:2]))
print(parsedate(IMAGES[0][0]))
np.vectorize(lambda iarr:np.array(parsedate(iarr)))(IMAGES[0])

['14_abril' '14_abril' '29_marzo' '29_marzo' '24_setiembre' '9_julio'
 '19_agosto' '2_julio' '23_julio' '5_agosto' '16_junio' '13_agosto'
 '16_junio' '14_julio' '26_mayo' '11_junio' '19_mayo' '26_mayo' '2_junio'
 '19_mayo' '28_abril' '7_mayo' '12_mayo' '12_mayo' '23_junio' '23_junio'
 '23_julio' '28_abril']


ValueError: setting an array element with a sequence

In [10]:
class Dataset_direct(torch.utils.data.Dataset):
    def __init__(self,root_dir,ImType=['PhantomRGB', 'SenteraRGB', 'SenteraNIR'],
                 Dates=None,
                 Trees=None,transform=None):
            """Data loader
            inputs:
            -root_dir(str): Directory that contains all the directories per tree
            -ImType(list of str): type of images to be used in posterior processing
                ['PhantomRGB', 'SenteraRGB', 'SenteraNIR']
            -Dates(list of str): Selected dates ex:['14_abril']
            -Trees(list of str): Selected Trees
            Outputs:
            -self.landmarks: Numpy of (nxm) where n is number of trees
            Troubleshooting:
            -if number of NIR images and RGB images from sentera do not match
            """
            'Initialization'
            
            Tr=os.listdir(root_dir)
            
            #Select per Trees
            if Trees!=None:
                inp=np.array(Trees)
                Tr=Tr[np.vectorize(lambda Tr,i:np.array(np.where(Tr==i)),signature="(j),()->()")(Tr,inp)]
            
            ImType=np.array(ImType)
            
            #Arrays of dates per trees
            IMAGES=np.vectorize(lambda DB,Tr,Type:np.array(os.listdir(os.path.join(root_dir,Tr,Type))),
             otypes=[object],
             signature="(),(),()->()")(DB,Tr,ImType.reshape(-1,1))
            
            
            self.landmarks_frame_Phantom = IMAGES[0]
            self.landmarks_frame_Sentera = IMAGES[1]
            
            
            #Select per dates
            if Dates!=None:
            
            
            self.transform = transform
            
            

    parsedate=np.vectorize(lambda iarr:("_").join(iarr.split("_")[:2]))
    def parse_dates():
    
    
    def __len__(self):
            'Denotes the total number of samples'
            return len(self.landmarks_frame)

    def __getitem__(self, idx):
            'Generates one sample of data'
            # Select sample
            if torch.is_tensor(idx):
                  idx=idx.tolist()

            img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[idx, 0])
            image = io.imread(img_name)
            landmarks = self.landmarks_frame.iloc[idx, 1:]
            landmarks = np.array([landmarks])

            landmarks = landmarks.astype('float').reshape(-1, 2)
            sample = {'image': image, 'landmarks': landmarks}
            if self.transform:
                  sample=self.transform(sample)

            return sample

In [45]:
np.vectorize?